# 顧客レビューを論点分解
- サーバレスコンピュートを使用します
- Kaggle [Airline Review](https://www.kaggle.com/datasets/chaudharyanshul/airline-reviews)のデータセットを使用します

注意：  
**日本語訳データをLLMで論点ごとに抽出＆要約しますが、1時間以上かかります。**  
**省略したい場合、`No.2　顧客レビューから論点抽出してサマライズ`はスキップして、`3. 論点ごとのサマリをポジ/ネガごとにレコード展開`に進んでください。**

#### 想定のディレクトリ構成
```
/<catalog_name>
├── airline_reviews                               <- スキーマ
│   ├── bz_reviews_en                             <- テーブル：bzonze/レビュー英語
│   ├── sv_reviews_ja                             <- テーブル：silver/レビュー日本語
│   ├── sv_reviews_summaries_by_topics            <- テーブル：silver/カテゴリ抽出＆要約
│   ├── gd_reviews_exstracted_by_categories_all   <- テーブル：gold/レコード展開
│   ├── raw_data                                  <- ボリューム
│       ├── EN/BA_AirlineReviews.csv              <- KaggleからダウンロードしたCSVを手動アップロード
│       ├── JA/BA_AirlineReviews.csv              <- 日本語翻訳データをCSV出力
```

#### 処理概要
1. レビュー内容を日本語翻訳
1. **レビュー内容からLLMで論点抽出＆要約 ←本ノートブック**
1. **論点ごとに抽出された要約文章をレコード展開 -> ダッシュボードで可視化 ←本ノートブック**

In [0]:
%run ./00_config

## 1. データ準備
日本語翻訳済みのCSVをVolumeから読み込みます

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

# define schema
schema = StructType([
    StructField("No", IntegerType(), True),
    StructField("OverallRating", StringType(), True),
    StructField("ReviewHeader", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Datetime", StringType(), True),
    StructField("VerifiedReview", StringType(), True),
    StructField("ReviewBody", StringType(), True),
    StructField("TypeOfTraveller", StringType(), True),
    StructField("SeatType", StringType(), True),
    StructField("Route", StringType(), True),
    StructField("DateFlown", StringType(), True),
    StructField("SeatComfort", StringType(), True),
    StructField("CabinStaffService", StringType(), True),
    StructField("GroundService", StringType(), True),
    StructField("ValueForMoney", StringType(), True),
    StructField("Recommended", StringType(), True),
    StructField("Aircraft", StringType(), True),
    StructField("FoodandBeverages", StringType(), True),
    StructField("InflightEntertainment", StringType(), True),
    StructField("WifiandConnectivity", StringType(), True)
])

# read files
df = spark.read.format("csv")\
                .option("header", True) \
                .option("multiLine", True) \
                .option("quote", '"') \
                .option("escape", '"') \
                .load(f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/JA/")

# write table
df.write.format("delta").mode("overwrite").saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_reviews_ja")

print(df.count())
print(df.columns)
display(df.limit(100))

## 2. 顧客レビューから論点抽出してサマライズ
作成テーブル：`sv_reviews_summaries_by_topics`  
処理：
今回扱う顧客レビューは設問設計がなされていない自由記述であり、複数要素が混在する。  
このままでは分析に使いにくいので、感情 x 論点ごとに要素分解する。  
分解要素は次のとおり。

- 感情
  - ポジティブ
  - ネガティブ
- 論点
  - 食事と飲み物に対する評価
  - 機内エンターテイメントに対する評価
  - 機内Wi-Fiと接続性に対する評価
  - 価格に対する価値の評価
  - 地上サービスに対する評価
  - 客室スタッフのサービスに対する評価
  - 座席の快適さに対する評価
  - その他


下記セル8-9は1時間以上かかります。ここでのLLM処理をスキップしたい場合、`3. 論点ごとのサマリをポジ/ネガごとにレコード展開`にショートカット可能です

In [0]:
import time

# タイマークラス
class TimeMeasurement:
    def __init__(self):
        self.start_time = None
        self.end_time = None

    def start(self):
        """タイマーを開始します。"""
        self.start_time = time.time()

    def stop(self):
        """タイマーを停止します。"""
        self.end_time = time.time()

    def elapsed_time(self):
        """経過時間を秒単位で返します。"""
        if self.start_time is None or self.end_time is None:
            raise ValueError("タイマーが開始または停止されていません。")
        return self.end_time - self.start_time

    def elapsed_time_formatted(self):
        """経過時間を HH:MM:SS 形式で返します。"""
        return time.strftime("%H:%M:%S", time.gmtime(self.elapsed_time()))

In [0]:
# 既存テーブル削除
spark.sql(f"DROP TABLE IF EXISTS {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")

# タイマー
timer = TimeMeasurement()

MAX_RECORDS = 1000      # 全体の処理件数
BATCH_SIZE = 10         # バッチサイズ

# 主キーの最小値と最大値を取得
min_max = spark.sql(f"SELECT MIN(No) AS min_no, MAX(No) AS max_no FROM {MY_CATALOG}.{MY_SCHEMA}.bz_reviews_ja").collect()[0]

min_no = int(min_max["min_no"])
max_no = int(min_max["max_no"])

# 処理件数が最大値を超えないように調整
if MAX_RECORDS > (max_no - min_no + 1):
    MAX_RECORDS = max_no - min_no + 1

# 範囲指定を使用したバッチ処理
for start_no in range(min_no, min_no + MAX_RECORDS, BATCH_SIZE):
    end_no = start_no + BATCH_SIZE - 1

    # タイマー開始
    timer = TimeMeasurement()
    timer.start()

    # プロンプト生成とAIクエリ実行
    batch_df = spark.sql(f"""
        WITH prompts AS (
          SELECT
            No,
            Name,
            Datetime,
            OverallRating,
            VerifiedReview,
            ReviewHeader,
            ReviewBody,
            ------------------------------
            -- 1. ポジティブ
            ------------------------------
            -- 1-1. ポジティブ_食事と飲み物
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ポジティブ、かつ「食事と飲み物」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「食事と飲み物」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS pos_food_and_beverages_prompt,

            -- 1-2. ポジティブ_機内エンターテイメント
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ポジティブ、かつ「機内エンターテイメント」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「機内エンターテイメント」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS pos_inflight_entertainment_prompt,

            -- 1-3. ポジティブ_機内Wi-Fiと接続性
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ポジティブ、かつ「機内Wi-Fiと接続性」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「機内Wi-Fiと接続性」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS pos_wifi_connectivity_prompt,

            -- 1-4. ポジティブ_価格に対する価値
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ポジティブ、かつ「価格に対する価値」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「価格に対する価値」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS pos_value_for_money_prompt,

            -- 1-5. ポジティブ_地上サービス
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ポジティブ、かつ「地上サービス」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「地上サービス」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS pos_ground_service_prompt,

            -- 1-6. ポジティブ_客室スタッフのサービス
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ポジティブ、かつ「客室スタッフのサービス」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「客室スタッフのサービス」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS pos_cabin_staff_service_prompt,

            -- 1-7. ポジティブ_座席の快適さ
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ポジティブ、かつ「座席の快適さ」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「座席の快適さ」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS pos_seat_comfort_prompt,

            -- 1-8. ポジティブ_その他
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ポジティブ、かつ次のいずれにも該当しない内容（「食事と飲み物」「機内エンターテイメント」「機内Wi-Fiと接続性」「価格に対する価値」「地上サービス」「客室スタッフのサービス」「座席の快適さ」）。",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「食事と飲み物」「機内エンターテイメント」「機内Wi-Fiと接続性」「価格に対する価値」「地上サービス」「客室スタッフのサービス」「座席の快適さ」など自明のものは外して良い）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS pos_others_prompt,

            ------------------------------
            -- 2. ネガティブ
            ------------------------------
            -- 2-1. ネガティブ_食事と飲み物
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ネガティブ、かつ「食事と飲み物」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「食事と飲み物」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS neg_food_and_beverages_prompt,

            -- 2-2. ネガティブ_機内エンターテイメント
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ネガティブ、かつ「機内エンターテイメント」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「機内エンターテイメント」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS neg_inflight_entertainment_prompt,

            -- 2-3. ネガティブ_機内Wi-Fiと接続性
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ネガティブ、かつ「機内Wi-Fiと接続性」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「機内Wi-Fiと接続性」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS neg_wifi_connectivity_prompt,

            -- 2-4. ネガティブ_価格に対する価値
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ネガティブ、かつ「価格に対する価値」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「価格に対する価値」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS neg_value_for_money_prompt,

            -- 2-5. ネガティブ_地上サービス
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ネガティブ、かつ「地上サービス」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「地上サービス」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS neg_ground_service_prompt,

            -- 2-6. ネガティブ_客室スタッフのサービス
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ネガティブ、かつ「客室スタッフのサービス」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「客室スタッフのサービス」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS neg_cabin_staff_service_prompt,

            -- 2-7. ネガティブ_座席の快適さ
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ネガティブ、かつ「座席の快適さ」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「座席の快適さ」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS neg_seat_comfort_prompt,

            -- 2-8. ネガティブ_その他
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              ReviewBody,
              "\n[抽出条件]ネガティブ、かつ次のいずれにも該当しない内容（「食事と飲み物」「機内エンターテイメント」「機内Wi-Fiと接続性」「価格に対する価値」「地上サービス」「客室スタッフのサービス」「座席の快適さ」）。",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「食事と飲み物」「機内エンターテイメント」「機内Wi-Fiと接続性」「価格に対する価値」「地上サービス」「客室スタッフのサービス」「座席の快適さ」など自明のものは外して良い）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください",
              "\n * レビュー対象である航空会社名（British Airways）は省くこと"
            ) AS neg_others_prompt,

            TypeOfTraveller,
            SeatType,
            Route,
            DateFlown,
            SeatComfort,
            CabinStaffService,
            GroundService,
            ValueForMoney,
            Recommended,
            Aircraft,
            FoodandBeverages,
            InflightEntertainment,
            WifiandConnectivity            
          FROM {MY_CATALOG}.{MY_SCHEMA}.bz_reviews_ja
          WHERE No BETWEEN {start_no} AND {end_no}
          ORDER BY No
        ),
        summaries_by_topics AS (
          -- exstruct summaries by topics
          SELECT
            No,
            Name,
            Datetime,
            OverallRating,
            VerifiedReview,
            ReviewHeader,
            ReviewBody,
            ------------------------------
            -- 1. ポジティブ
            ------------------------------
            -- 1-1. ポジティブ_食事と飲み物に対する評価
            pos_food_and_beverages_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_food_and_beverages_prompt, failOnError => False).result AS pos_food_and_beverages_result,

            -- 1-2. ポジティブ_機内エンターテイメントに対する評価
            pos_inflight_entertainment_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_inflight_entertainment_prompt, failOnError => False).result AS pos_inflight_entertainment_result,

            -- 1-3. ポジティブ_機内Wi-Fiと接続性に対する評価
            pos_wifi_connectivity_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_wifi_connectivity_prompt, failOnError => False).result AS pos_wifi_connectivity_result,

            -- 1-4. ポジティブ_価格に対する価値の評価
            pos_value_for_money_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_value_for_money_prompt, failOnError => False).result AS pos_value_for_money_result,

            -- 1-5. ポジティブ_地上サービスに対する評価
            pos_ground_service_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_ground_service_prompt, failOnError => False).result AS pos_ground_service_result,

            -- 1-6. ポジティブ_客室スタッフのサービスに対する評価
            pos_cabin_staff_service_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_cabin_staff_service_prompt, failOnError => False).result AS pos_cabin_staff_service_result,

            -- 1-7. ポジティブ_座席の快適さに対する評価
            pos_seat_comfort_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_seat_comfort_prompt, failOnError => False).result AS pos_seat_comfort_result,

            -- 1-8. ポジティブ_その他
            pos_others_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_others_prompt, failOnError => False).result AS pos_others_result,

            ------------------------------
            -- 2. ネガティブ
            ------------------------------
            -- 2-1. ネガティブ_食事と飲み物に対する評価
            neg_food_and_beverages_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_food_and_beverages_prompt, failOnError => False).result AS neg_food_and_beverages_result,

            -- 2-2. ネガティブ_機内エンターテイメントに対する評価
            neg_inflight_entertainment_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_inflight_entertainment_prompt, failOnError => False).result AS neg_inflight_entertainment_result,

            -- 2-3. ネガティブ_機内Wi-Fiと接続性に対する評価
            neg_wifi_connectivity_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_wifi_connectivity_prompt, failOnError => False).result AS neg_wifi_connectivity_result,

            -- 2-4. ネガティブ_価格に対する価値の評価
            neg_value_for_money_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_value_for_money_prompt, failOnError => False).result AS neg_value_for_money_result,

            -- 2-5. ネガティブ_地上サービスに対する評価
            neg_ground_service_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_ground_service_prompt, failOnError => False).result AS neg_ground_service_result,

            -- 2-6. ネガティブ_客室スタッフのサービスに対する評価
            neg_cabin_staff_service_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_cabin_staff_service_prompt, failOnError => False).result AS neg_cabin_staff_service_result,

            -- 2-7. ネガティブ_座席の快適さに対する評価
            neg_seat_comfort_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_seat_comfort_prompt, failOnError => False).result AS neg_seat_comfort_result,

            -- 2-8. ネガティブ_その他
            neg_others_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_others_prompt, failOnError => False).result AS neg_others_result,

            TypeOfTraveller,
            SeatType,
            Route,
            DateFlown,
            SeatComfort,
            CabinStaffService,
            GroundService,
            ValueForMoney,
            Recommended,
            Aircraft,
            FoodandBeverages,
            InflightEntertainment,
            WifiandConnectivity
          FROM prompts
        )
        SELECT
          No,
          Name,
          Datetime,
          OverallRating,
          VerifiedReview,
          ReviewHeader,
          ReviewBody,
          
          ------------------------------
          -- 1. ポジティブ
          ------------------------------
          -- 1-1. ポジティブ_食事と飲み物に対する評価
          pos_food_and_beverages_prompt,
          COALESCE(pos_food_and_beverages_result, '-') AS pos_food_and_beverages_summary,

          -- 1-2. ポジティブ_機内エンターテイメントに対する評価
          pos_inflight_entertainment_prompt,
          COALESCE(pos_inflight_entertainment_result, '-') AS pos_inflight_entertainment_summary,

          -- 1-3. ポジティブ_機内Wi-Fiと接続性に対する評価
          pos_wifi_connectivity_prompt,
          COALESCE(pos_wifi_connectivity_result, '-') AS pos_wifi_connectivity_summary,

          -- 1-4. ポジティブ_価格に対する価値の評価
          pos_value_for_money_prompt,
          COALESCE(pos_value_for_money_result, '-') AS pos_value_for_money_summary,

          -- 1-5. ポジティブ_地上サービスに対する評価
          pos_ground_service_prompt,
          COALESCE(pos_ground_service_result, '-') AS pos_ground_service_summary,

          -- 1-6. ポジティブ_客室スタッフのサービスに対する評価
          pos_cabin_staff_service_prompt,
          COALESCE(pos_cabin_staff_service_result, '-') AS pos_cabin_staff_service_summary,

          -- 1-7. ポジティブ_座席の快適さに対する評価
          pos_seat_comfort_prompt,
          COALESCE(pos_seat_comfort_result, '-') AS pos_seat_comfort_summary,

          -- 1-8. ポジティブ_その他
          pos_others_prompt,
          COALESCE(pos_others_result, '-') AS pos_others_summary,

          ------------------------------
          -- 2. ネガティブ
          ------------------------------
          -- 2-1. ネガティブ_食事と飲み物に対する評価
          neg_food_and_beverages_prompt,
          COALESCE(neg_food_and_beverages_result, '-') AS neg_food_and_beverages_summary,

          -- 2-2. ネガティブ_機内エンターテイメントに対する評価
          neg_inflight_entertainment_prompt,
          COALESCE(neg_food_and_beverages_result, '-') AS neg_inflight_entertainment_summary,

          -- 2-3. ネガティブ_機内Wi-Fiと接続性に対する評価
          neg_wifi_connectivity_prompt,
          COALESCE(neg_wifi_connectivity_result, '-') AS neg_wifi_connectivity_summary,

          -- 2-4. ネガティブ_価格に対する価値の評価
          neg_value_for_money_prompt,
          COALESCE(neg_value_for_money_result, '-') AS neg_value_for_money_summary,

          -- 2-5. ネガティブ_地上サービスに対する評価
          neg_ground_service_prompt,
          COALESCE(neg_ground_service_result, '-') AS neg_ground_service_summary,

          -- 2-6. ネガティブ_客室スタッフのサービスに対する評価
          neg_cabin_staff_service_prompt,
          COALESCE(neg_cabin_staff_service_result, '-') AS neg_cabin_staff_service_summary,

          -- 2-7. ネガティブ_座席の快適さに対する評価
          neg_seat_comfort_prompt,
          COALESCE(neg_seat_comfort_result, '-') AS neg_seat_comfort_summary,

          -- 2-8. ネガティブ_その他
          neg_others_prompt,
          COALESCE(neg_others_result, '-') AS neg_others_summary,

          TypeOfTraveller,
          SeatType,
          Route,
          DateFlown,
          SeatComfort,
          CabinStaffService,
          GroundService,
          ValueForMoney,
          Recommended,
          Aircraft,
          FoodandBeverages,
          InflightEntertainment,
          WifiandConnectivity
        FROM summaries_by_topics
    """)

    # Deltaテーブルへのインクリメンタル書き込み
    (batch_df.write.format("delta")
        .mode("append")                 # 既存データに追記
        .option("mergeSchema", "true")  # スキーマ変更を許容
        .saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics"))

    # タイマー停止
    timer.stop()

    # 所要時間の表示（秒単位とフォーマット済み）
    elapsed_seconds = timer.elapsed_time()
    elapsed_formatted = timer.elapsed_time_formatted()

    print(f"Processed range: {start_no + 1} - {end_no + 1} （所要時間: {elapsed_formatted} / {elapsed_seconds:.2f} 秒）")

In [0]:
df = spark.table(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")
display(
  df.select(
    'No',
    'OverallRating',
    'ReviewBody',
    'pos_food_and_beverages_summary',
    'pos_inflight_entertainment_summary',
    'pos_wifi_connectivity_summary',
    'pos_value_for_money_summary',
    'pos_ground_service_summary',
    'pos_cabin_staff_service_summary',
    'pos_seat_comfort_summary',
    'pos_others_summary',
    'neg_food_and_beverages_summary',
    'neg_inflight_entertainment_summary',
    'neg_wifi_connectivity_summary',
    'neg_value_for_money_summary',
    'neg_ground_service_summary',
    'neg_cabin_staff_service_summary',
    'neg_seat_comfort_summary',
    'neg_others_summary',
    'TypeOfTraveller',
    'SeatType',
    'Route',
    'DateFlown',
    'SeatComfort',
    'CabinStaffService',
    'GroundService',
    'ValueForMoney',
    'Recommended',
    'Aircraft',
    'FoodandBeverages',
    'InflightEntertainment',
    'WifiandConnectivity'
  )
)

## 3. 論点ごとのサマリをポジ/ネガごとにレコード展開
作成テーブル：`gd_reviews_exstracted_by_categories_all`
処理：以下の論点ごとに横方向（カラム）に分かれているが、このままだと使いにくいので、縦方向（レコード）に展開する。  
処理のイメージ
```
会員ID  | 機内スタッフの接客の質     | 食事や飲み物の品質      | 座席の快適さ      | 地上サービスの質      | 価格に対する価値
A001   | *****                   | *****               | *****           | *****               | *****
A002   | *****                   | *****               | *****           | *****               | *****

↓

A001   | 機内スタッフのサービス| *****
A001   | 食事や飲み物        | *****
A001   | 座席の快適さ        | *****
A001   | 地上サービス        | *****
A001   | 価格に対する価値     | *****
A001   | その他             | *****
A002   | 機内スタッフのサービス| *****
A002   | 食事や飲み物        | *****
A002   | 座席の快適さ        | *****
A002   | 地上サービス        | *****
A002   | 価格に対する価値     | *****
A002   | その他             | *****
```

**【チート処理】**
`2. 顧客レビューから論点抽出してサマライズ`をスキップした場合、次のセル13のコメントアウトを外して実行してください。

本来はLLMで処理してテーブル`sv_reviews_summaries_by_topics`を作りますが、  
代わりにVolumeからCSVデータを読み込んでテーブル作成することで、LLM処理の待機時間をショートカットします。

In [0]:
# # read files
# df = spark.read.format("csv")\
#                 .option("header", True) \
#                 .option("multiLine", True) \
#                 .option("quote", '"') \
#                 .option("escape", '"') \
#                 .load(f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/extracted/")

# # write table
# df.write.format("delta").mode("overwrite").saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")

# print(df.count())
# print(df.columns)
# display(df.limit(100))

In [0]:
from pyspark.sql.types import StructType, StructField, LongType, StringType, DateType, IntegerType, BooleanType
from pyspark.sql.functions import explode, struct, lit, col, array

# データフレームの読み込み
df = spark.read.table(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")

# スキーマ定義
schema = StructType([
    StructField("No", LongType(), True),
    StructField("Name", StringType(), True),
    StructField("Datetime", DateType(), True),
    StructField("OverallRating", IntegerType(), True),
    StructField("VerifiedReview", BooleanType(), True),
    StructField("ReviewHeader", StringType(), True),
    StructField("ReviewBody", StringType(), True),
    StructField("review_category", StringType(), True),     # レビューカテゴリ
    StructField("review_details", StringType(), True),      # レビューサマリ
    StructField("TypeOfTraveller", StringType(), True),
    StructField("SeatType", StringType(), True),
    StructField("Route", StringType(), True),
    StructField("DateFlown", StringType(), True),           # 例: "2023-11"
    StructField("SeatComfort", IntegerType(), True),
    StructField("CabinStaffService", IntegerType(), True),
    StructField("GroundService", IntegerType(), True),
    StructField("ValueForMoney", IntegerType(), True),
    StructField("Recommended", StringType(), True),         # お勧めするか（yes/no）
    StructField("Aircraft", StringType(), True),
    StructField("FoodandBeverages", IntegerType(), True),
    StructField("InflightEntertainment", IntegerType(), True),
    StructField("WifiandConnectivity", IntegerType(), True),
])

# 構造体配列の作成
df_with_arrays = df.withColumn(
    "review_pos",
    array(
        struct(lit("食事と飲み物").alias("category"), col("pos_food_and_beverages_summary").alias("review")),
        struct(lit("機内エンターテイメント").alias("category"), col("pos_inflight_entertainment_summary").alias("review")),
        struct(lit("機内Wi-Fiと接続性").alias("category"), col("pos_wifi_connectivity_summary").alias("review")),
        struct(lit("価格に対する価値").alias("category"), col("pos_value_for_money_summary").alias("review")),
        struct(lit("地上サービス").alias("category"), col("pos_ground_service_summary").alias("review")),
        struct(lit("機内スタッフのサービス品質").alias("category"), col("pos_cabin_staff_service_summary").alias("review")),
        struct(lit("座席の快適さ").alias("category"), col("pos_seat_comfort_summary").alias("review")),
        struct(lit("その他").alias("category"), col("pos_others_summary").alias("review"))
    )
).withColumn(
    "review_neg",
    array(
        struct(lit("食事と飲み物").alias("category"), col("neg_food_and_beverages_summary").alias("review")),
        struct(lit("機内エンターテイメント").alias("category"), col("neg_inflight_entertainment_summary").alias("review")),
        struct(lit("機内Wi-Fiと接続性").alias("category"), col("neg_wifi_connectivity_summary").alias("review")),
        struct(lit("価格に対する価値").alias("category"), col("neg_value_for_money_summary").alias("review")),
        struct(lit("地上サービス").alias("category"), col("neg_ground_service_summary").alias("review")),
        struct(lit("機内スタッフのサービス品質").alias("category"), col("neg_cabin_staff_service_summary").alias("review")),
        struct(lit("座席の快適さ").alias("category"), col("neg_seat_comfort_summary").alias("review")),
        struct(lit("その他").alias("category"), col("neg_others_summary").alias("review"))
    )
)

# カラム順序制御用の定義
main_columns = [
    "No", "Name", "Datetime", "OverallRating", "VerifiedReview", "ReviewHeader", "ReviewBody"
]

additional_columns = [
    "TypeOfTraveller", "SeatType", "Route", "DateFlown", "SeatComfort", "CabinStaffService", "GroundService",
    "ValueForMoney", "Recommended", "Aircraft", "FoodandBeverages", "InflightEntertainment", "WifiandConnectivity"
]

# ポジティブ展開処理
df_exploded_pos = df_with_arrays.select(
    *main_columns,
    explode("review_pos").alias("temp_data"),
    *additional_columns
).select(
    *main_columns,
    col("temp_data.category").alias("review_category"),
    col("temp_data.review").alias("review_details"),
    *additional_columns
).filter(col("review_details") != "-")

# ネガティブ展開処理
df_exploded_neg = df_with_arrays.select(
    *main_columns,
    explode("review_neg").alias("temp_data"),
    *additional_columns
).select(
    *main_columns,
    col("temp_data.category").alias("review_category"),
    col("temp_data.review").alias("review_details"),
    *additional_columns
).filter(col("review_details") != "-")

# データ型変換（全カラムに適用）
for field in schema.fields:
    col_name = field.name
    dtype = field.dataType
    df_exploded_pos = df_exploded_pos.withColumn(col_name, col(col_name).cast(dtype))
    df_exploded_neg = df_exploded_neg.withColumn(col_name, col(col_name).cast(dtype))

# 変換後のDataFrameを直接使用
df_exploded_pos_converted = df_exploded_pos
df_exploded_neg_converted = df_exploded_neg

# create tempview
df_exploded_pos_converted.createOrReplaceTempView("sv_reviews_exstracted_by_categories_pos")
df_exploded_neg_converted.createOrReplaceTempView("sv_reviews_exstracted_by_categories_neg")

# UNION
df = spark.sql('''
SELECT *, "positive" as sentiment FROM sv_reviews_exstracted_by_categories_pos
UNION ALL
SELECT *, "negative" as sentiment FROM sv_reviews_exstracted_by_categories_neg
''')

# write table
df.write.format("delta").mode("overwrite").saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.gd_reviews_exstracted_by_categories_all")

print(df.count())
print(df.columns)
display(df.limit(100))

In [0]:
from pyspark.sql.functions import col

df = spark.table(f"{MY_CATALOG}.{MY_SCHEMA}.gd_reviews_exstracted_by_categories_all")

'''
概要：llmが生成したサマリ文章に余計な文字列がも含まれていないかチェックする
処理：llmが生成したサマリから" -"が含まれるレコードを抽出
'''
# df_bad = df.filter(col("review_details") != "-")
df_bad = df.filter(col("review_details").contains(" -"))

print(df_bad.count())
print(df_bad.columns)
display(df_bad.select("review_details").distinct())